# Simple Nearest Neighbours document classification

In this exercise, you will write a program that can classify documents into any number of classes, based on provided training data. There are many classification algorithms; we will look at one of the simplest possible methods, that still often turns out to work quite well in practice: the *nearest neighbour classifier*.

The idea is as follows:
1. We define a distance function between documents. The distance between two documents that are equal is zero. The more different the documents are, the larger the distance.
2. Given a new document that we want to classify, we calculate the distance between it and *all* documents in the training data. One of them will be closest according to the distance measure we defined above. We report the class of the closest match as the predicted class for the new document.

You can test your program on the spam dataset that is provided via blackboard. We provide lots of hints as to how to process your data and how to set up your distance function; but some parts of the design will be yours to choose.

---

### Exercise 1


We provided some code that allows you to read in an email document. The first step is to be able to figure out all the files that are in some directory.

Try out the filelist function below to read and print the files in one of the email directories.

---

In [99]:
from os import listdir           # used to see what files are in a directory
from os.path import isfile, join # join concatenates parts of a pathname

# in:  the pathname of a directory
# out: the list of files in that directory
def filelist(path):
    return [f for f in listdir(path) if isfile(join(path, f))]

In [100]:
#filelist("C:/Users/ntb/Desktop/LEIDEN Ucebne Materialy/Introduction to data science/week3Python/data/spam/test")

The next step is to read all the contents of a file into a Python string.

In [101]:
# in:  the pathname of a directory, and the name of a file in that directory
# out: a string containing the contents of the file
def read_file(path, name):
    handle = open(join(path, name), "rb")#rb means that it will treat emails as binary
    res = handle.read()
    handle.close()
    return res

In [102]:
string_file=read_file("C:/Users/ntb/Desktop/LEIDEN Ucebne Materialy/Introduction to data science/week3Python/data/test/spam",'0006.7a32642f8c22bbeb85d6c3b5f3890a2c' )
string_file = str(string_file)
#print(string_file)

---

### Exercise 2

To simplify our task, we are going to ignore the word order and word frequencies in the email messages. To do so, modify the function below so it converts each document string into a *set of words*. Include only words that consist solely of alphabetic characters, and convert all words to lower case.

---

In [103]:
# in:  a long string containing a document
# out: a (python) set containing all words in the document, in lower case.
def extract_words(file):
    import re
    new_file = re.sub("[^a-zA-Z ]+", "", file)
    new_file_lower=str.lower(new_file)
    wordList = re.sub("[^\w]", " ",  new_file_lower).split()
    return(list(set(wordList)))

In [104]:
#extract_words(string_file)

---

### Exercise 3

Now that we can read word sets, it is time to decide on an appropriate distance function. We leave the definition of this function up to you: adapt the skeleton code below into a function that you believe will work well, and briefly motivate your choice (in perhaps two lines of text).

---

In [105]:
# in:  two word sets
# out: their distance (0 if the sets are equal)
#think about normalization of distance for example divinding by measure of email 
#short emails will always have shortest distance
     
def distance(wordset1, wordset2):
    common_list=[element for element in wordset1 if element in wordset2]
    distance=(len(wordset1)-len(common_list))/len(wordset1)
    return distance
#     if wordset1 in string_file and wordset2 in string_file:
#        return abs(words.index(wordset1) - words.index(wordset2))

In [106]:
distance("only", "remove")


0.75

** Motivation: **

---

### Exercise 4

We can now compare individual documents. However, we do not have any code for working with all documents in a class simultaneously. A class will be represented as a *dictionary*, mapping the name of each file it contains to the set of words in that file.


Adapt the read_class function below to create the dictionary given the name of a directory containing the files for that class.

---

In [107]:
# in:  the pathname of a directory containing all files that describe a class
# out: a dictionary mapping each filename to its word set
#call it twice
def read_class(path):
 file_list=filelist(path)
 import re 
 my_dictionary={}
 for file in file_list:
    string_file=str(read_file(path,file))
    my_dictionary[file]=extract_words(string_file)
 return my_dictionary    

In [108]:
path="C:/Users/ntb/Desktop/LEIDEN Ucebne Materialy/Introduction to data science/week3Python/data/train/spam"
#read_class(path)

---

### Exercise 5

Now that we can obtain the representation of a training class, we need to be able to find how well a test document matches against that class.

To do so, the function below should look for the *best matching document* in the class, and return its name and its distance.

---

In [109]:
# in:  a wordset and a class
# out: the tuple (f,d) where f is the filename of the best match, and
#                            d is its distance to the given wordset.
#class means probably a file
#one possible way..attach to dictionary another information about distance and then 
#take minimum of distance...name of false should be also in dictionary
def best_match(wordset, cls):
    class_files=filelist(path)
    cls=read_class(path)
    dist={}
    for file in class_files:
     dist[file]=distance(wordset,cls[file]) 
    minimum=min(dist.items(), key=lambda x: x[1]) 
    return(minimum)#gives name of fale and distance to nearest neighboor in that class
        

In [110]:
wordset=["yes","and", "something"]
best_match(wordset,path)       

('0446.d971f234fa00ed94e18f5ce8f5c0f852', 0.3333333333333333)

---

### Exercise 6

This is where it gets a little bit abstract. Just like we have created classes, that are dictionaries which map filenames to wordsets, it will be convenient to bunch up all training classes into a single dictionary, that maps the name of the class to the data for that class.

---

In [111]:
# in:  the names of all the considered classes
# out: a dictionary mapping each class name to its data
#class names spam and ham?
def read_training_data(class_names):
    class_dict={}
    path="C:/Users/ntb/Desktop/LEIDEN Ucebne Materialy/Introduction to data science/week3Python/data/train"
    s="/" 
    for c in class_names:
     seq=(path,c)
     path1=s.join(seq)
     class_dict[c]=read_class(path1)
    return(class_dict)

In [112]:
class_names=["spam","ham"]
#read_training_data(class_names)


---

### Exercise 7

It is now finally time to implement the nearest neighbour algorithm. The function below should find the best match for the given wordset in *each* class of the training data, and return which file matched best, at what distance that file matched, and what class it belongs to. That class will be the predicted class for the given wordset.

---

In [113]:
# in:  a wordset and a dict of classes
# out: the filename of the closest file, the distance to that file, and the name of its class
def nearest_neighbour(wordset, training_data):
    dis={}
    for clas in training_data.keys():
        dis[clas]=best_match(wordset,training_data[clas]) 
    minimum=min(dis.items(), key=lambda x: x[1]) 
    return(minimum)

In [114]:
training_data=read_training_data(class_names)
nearest_neighbour(wordset,training_data)

#training_data["spam"]["0000.7b1b73cf36cf9dbc3d64e3f2ee2b91f1"]


('ham', ('0446.d971f234fa00ed94e18f5ce8f5c0f852', 0.3333333333333333))

---

### Exercise 8

Write a test function that will run through all the files in a test directory and predict their class based on the nearest neighbour algorithm using the training data. Report which files get misclassified and at the end report what fraction of files are classified correctly.

Also write a main function that applies the test to all test directories. Make sure it works when we run it!

The sensitivity of the classifier is the fraction of test spam mails that are correctly classified. The specificity is the fraction of test ham mails that are correctly classified. Do these scores turn out similar for you? Why or why not?

The best scores for sensitivity and specificity will be reported in class :-)

---

In [80]:
# in:  a dict of training classes, and the name of the class to test
# out: a report of the test success
def test(training_data, test_class_name):
    print("Testing ", test_class_name)
    path="C:/Users/ntb/Desktop/LEIDEN Ucebne Materialy/Introduction to data science/week3Python/data/test"
    s="/"
    path1=s.join(seq("path",test_class_name))
    clas_test=read_class(path1)
    dic={}
    for file in clas_test:
       dic[file]=nearest_neighbour(file,training_data)[0]#assigns class to file
def main():
    None # Adapt

SyntaxError: invalid syntax (<ipython-input-80-f6f10aaf27fa>, line 12)

In [ ]:
#test(training_data,"spam")
#test_class_name="spam"
#path11="C:/Users/ntb/Desktop/LEIDEN Ucebne Materialy/Introduction to data science/week3Python/data/test"
#s="/"
#seq=(path,test_class_name)
pathh="C:/Users/ntb/Desktop/LEIDEN Ucebne Materialy/Introduction to data science/week3Python/data/test/spam"
clas_test1=read_class(pathh)
dic={}
#for file in clas_test.keys():
#    dic[file]=nearest_neighbour(clas_test[file],training_data)[0]#assigns class to file
#print(dic)

print(clas_test1)
